In [2]:
from ray.tune.integration.mlflow import mlflow_mixin 
import mlflow
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune

In [ ]:
# export env variable to access the port when using docker. 
# otherwise, use the default mlflow 

In [ ]:
# Ray tune need a trainable function to pass to tune.run
# A required input is config.
# Ray Tune provides a wrapper function, 
# called tune.with_parameters, 
# which allows you to pass along additional 
# arbitrary parameters and objects


@mlflow_mixin
def finetuning_dl_model(config, data_dir = None, num_epochs = 3, num_gpus = 0):
    mlflow.pytorch.autolog()
    # datamodule: batch_size tuned
    datamodule = ImageClassificationData.from_files()

    # classifier_model: optimizer, lr, backbone tuned
    classifier_model = ImageClassifier()    

    metrics = {"loss": "val_cross_entropy", "f1": "val_f1"}

    # Tune Ray reports the score of metrics after every trials
    trainer = flash.Trainer(max_epochs = num_epochs,
                            gpus = num_gpus,
                            progress_bar_refresh_rate=0, 
                            callbacks = [TuneReportCallback(metrics,on= 'validation_end')])
    trainer.finetune(classifier_model, datamodule, strategy= config['finetuning_strategies'])

    # batch_size hyperparameter is not automatically captured by autologging
    mlflow.log_param('batch_size', config['batch_size'])
    mlflow.log_param('back_bone', config['back_bone'])
    # ... 

In [ ]:
# pass the trainable function to tune.with_parameters



def run_hpo_dl_model(num_samples=10,
                     num_epochs=3,
                     gpus_per_trial=0,
                     tracking_uri=None,
                     experiment_name="hpo-tuning"):
    
    data_dir = 'data'

    # Set the MLflow experiment, or create it if it does not exist.
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment_name)

    # create config
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([1, 3, 6]),
        "foundation_model": "resnet50",
        "finetuning_strategies": "freeze",
        "optimizer_type": "Adam",
        "mlflow": {
            "experiment_name": experiment_name,
            "tracking_uri": mlflow.get_tracking_uri()
        }
    }
    
    # Create the trainable function to pass to tune.run() - avtivate HPO Ray tune
    trainable = tune.with_parameters(finetuning_dl_model,data_dir,num_epochs, gpus_per_trial)

    analysis = tune.run(
        trainable,
        resources_per_trial={
            "cpu": 1,
            "gpu": gpus_per_trial
        },
        metric="f1",
        mode="max",
        config=config,
        num_samples=num_samples,
        name="hpo_tuning_dl_model")
    
    print("Best hyperparameters found were: %s", analysis.best_config)



